In [1]:
# Carga de librerias
import pandas as pd
import numpy as np 

In [2]:
common_player_info = pd.read_csv("common_player_info.csv")
player = pd.read_csv("player.csv")
df_salarios = pd.read_csv("salarios_nba_2013_2023.csv")
mvp_final = pd.read_csv("mvps.csv")


In [9]:
!pip install unidecode



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import unidecode

def normalizar_nombre(nombre):
    if pd.isna(nombre):
        return ""
    return unidecode.unidecode(nombre.lower().strip())


In [11]:
player['clean_name'] = player['full_name'].apply(normalizar_nombre)
df_salarios['clean_name'] = df_salarios['jugador'].apply(normalizar_nombre)
mvp_final['clean_name'] = mvp_final['Player'].apply(normalizar_nombre)


In [12]:
# En df_salarios la columna es tipo "2013-2014", tomamos el primer año
df_salarios['año'] = df_salarios['temporada'].str[:4].astype(int)

# En mvp_final la columna se llama Year
mvp_final['año'] = mvp_final['Year'].astype(int)


In [14]:
players_salarios = player.merge(df_salarios, on='clean_name', how='left')


In [15]:
df_final = players_salarios.merge(mvp_final, on=['clean_name', 'año'], how='left')


In [16]:
df_final.to_csv("jugadores_completo.csv", index=False)


In [17]:
players_salarios.to_csv("jugadores_con_salario.csv", index=False)

In [18]:
print("Jugadores únicos en players:", player['clean_name'].nunique())
print("Jugadores únicos en salarios:", df_salarios['clean_name'].nunique())


Jugadores únicos en players: 4791
Jugadores únicos en salarios: 1450


In [21]:
jugadores_unicos = set(player['clean_name'].unique())
salarios_unicos = set(df_salarios['clean_name'].unique())

jugadores_sin_salario = jugadores_unicos - salarios_unicos

print("Jugadores sin salario:", len(jugadores_sin_salario))



Jugadores sin salario: 3497


In [22]:
salarios_sin_player = salarios_unicos - jugadores_unicos
print("Jugadores con salario pero sin estar en tabla players:", len(salarios_sin_player))


Jugadores con salario pero sin estar en tabla players: 156


In [23]:
coinciden = jugadores_unicos & salarios_unicos
print("Jugadores en común:", len(coinciden))


Jugadores en común: 1294


In [26]:
list(salarios_sin_player)[:20]


['justin minaya',
 'anthony barber',
 'james webb',
 'cody demps',
 'michael frazier',
 'trey murphy',
 'vernon carey',
 'dennis schroeder',
 'roger mason',
 'lonnie walker',
 'wendell carter',
 'scott wood',
 'chaundee brown',
 'aric holman',
 'derrick jones',
 'james ennis',
 'xavier cooks',
 'brian bowen',
 'perry jones',
 'landry nnoko']

In [4]:
import unidecode

def normalizar_nombre(nombre):
    if pd.isna(nombre):
        return ""
    return unidecode.unidecode(nombre.lower().strip())

# En common_player_info
common_player_info['clean_name'] = common_player_info['display_first_last'].apply(normalizar_nombre)

# En df_salarios
df_salarios['clean_name'] = df_salarios['jugador'].apply(normalizar_nombre)


In [6]:
df_salarios_merged = df_salarios.merge(
    common_player_info[['person_id', 'display_first_last', 'clean_name']],
    on='clean_name',
    how='left'
)


In [11]:
salarios_pivot = df_salarios_merged.pivot_table(
    index=['person_id', 'display_first_last'],
    columns='temporada',
    values='salario',
    aggfunc='first'
).reset_index()


In [12]:
print(df_salarios_merged.columns)


Index(['temporada', 'jugador', 'salario', 'clean_name', 'person_id',
       'display_first_last'],
      dtype='object')


In [13]:
print(salarios_pivot)

temporada  person_id display_first_last   2013-2014   2014-2015  2015-2016  \
0              199.0    Reggie Williams    111518.0    487905.0        NaN   
1              708.0      Kevin Garnett  11532468.0  12000000.0  8500000.0   
2              948.0       Marcus Camby   3383773.0   8354416.0        NaN   
3              959.0         Steve Nash   9300500.0   9701000.0        NaN   
4              965.0       Derek Fisher   1399507.0         NaN        NaN   
...              ...                ...         ...         ...        ...   
1128       1631298.0         Jack White         NaN         NaN        NaN   
1129       1631306.0        Cole Swider         NaN         NaN        NaN   
1130       1631311.0    Lester Quinones         NaN         NaN        NaN   
1131       1631320.0       Chima Moneke         NaN         NaN        NaN   
1132       1631323.0  Simone Fontecchio         NaN         NaN        NaN   

temporada  2016-2017  2017-2018  2018-2019  2019-2020  2020-202

In [14]:
salarios_pivot.to_csv("salarios_pivot.csv", index=False)


In [16]:
print("Total de jugadores en common_player_info:", common_player_info['person_id'].nunique())


Total de jugadores en common_player_info: 4171


In [18]:
print("Jugadores con salario después del pivot:", salarios_pivot['person_id'].nunique())


Jugadores con salario después del pivot: 1133


In [19]:
import unidecode
import pandas as pd

# 1. Función para normalizar nombres
def normalizar_nombre(nombre):
    if pd.isna(nombre):
        return ""
    return unidecode.unidecode(nombre.lower().strip())

# 2. Normalizamos los nombres
common_player_info['clean_name'] = common_player_info['display_first_last'].apply(normalizar_nombre)
df_salarios['clean_name'] = df_salarios['jugador'].apply(normalizar_nombre)

# 3. Hacemos merge LEFT para mantener todos los jugadores
merged = common_player_info[['person_id', 'display_first_last', 'clean_name']].merge(
    df_salarios[['clean_name', 'temporada', 'salario']],
    on='clean_name',
    how='left'
)

# 4. Pivotamos: filas = jugador, columnas = temporada, valores = salario
salarios_final = merged.pivot_table(
    index=['person_id', 'display_first_last'],
    columns='temporada',
    values='salario',
    aggfunc='first'
).reset_index()

# 5. (opcional) Ordenar columnas para que las temporadas estén en orden
columnas_ordenadas = ['person_id', 'display_first_last'] + sorted([c for c in salarios_final.columns if c not in ['person_id', 'display_first_last']])
salarios_final = salarios_final[columnas_ordenadas]


In [20]:
salarios_final.to_csv("salarios_final.csv", index=False)


In [21]:
df_salarios['jugador'].nunique()


1450

In [22]:
salarios_final['display_first_last'].nunique()

1121

In [24]:
common_player_info['clean_name'] = common_player_info['display_first_last'].apply(normalizar_nombre)
df_salarios['clean_name'] = df_salarios['jugador'].apply(normalizar_nombre)


In [25]:
merged = common_player_info[['person_id', 'display_first_last', 'clean_name']].merge(
    df_salarios[['clean_name', 'temporada', 'salario']],
    on='clean_name',
    how='left'
)


In [26]:
salarios_final = merged.pivot_table(
    index=['person_id', 'display_first_last'],
    columns='temporada',
    values='salario',
    aggfunc='first'
).reset_index()


In [27]:
# Normalizamos en salarios_final
salarios_final['clean_name'] = salarios_final['display_first_last'].apply(normalizar_nombre)

# Conjuntos
nombres_salario = set(df_salarios['clean_name'].unique())
nombres_final = set(salarios_final['clean_name'].unique())

# Diferencias
nombres_perdidos = nombres_salario - nombres_final
print("Jugadores con salario que no lograron unirse:", len(nombres_perdidos))


Jugadores con salario que no lograron unirse: 329


In [28]:
!pip install rapidfuzz


   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 14.9 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
from rapidfuzz import process, fuzz

# Lista de nombres desde common_player_info ya normalizados
nombres_base = common_player_info['clean_name'].unique().tolist()

# Filtramos sólo los nombres que no lograron unirse
nombres_perdidos_lista = list(nombres_perdidos)

# Creamos sugerencias automáticas
sugerencias = []

for nombre in nombres_perdidos_lista:
    match, score, _ = process.extractOne(
        nombre,
        nombres_base,
        scorer=fuzz.token_sort_ratio
    )
    if score > 90:  # podemos ajustar este umbral si hace falta
        sugerencias.append((nombre, match, score))

# Convertimos a DataFrame para ver
df_sugerencias = pd.DataFrame(sugerencias, columns=['nombre_salario', 'match_common', 'score'])
df_sugerencias.head(20)


,nombre_salario,match_common,score
0,mckinley wright,mckinley wright iv,90.909091
1,michael frazier,michael frazier ii,90.909091
2,vince williams jr,vince williams jr.,97.142857
3,dennis schroeder,dennis schroder,96.774194
4,larry nance jr,larry nance jr.,96.551724
5,vince hunter,vincent hunter,92.307692
6,keldon johnson,keon johnson,92.307692
7,toure murry,toure' murry,95.652174
8,louis williams,lou williams,92.307692
9,ja'vonte smart,javonte smart,96.296296


In [31]:
df_correcciones = df_sugerencias[df_sugerencias['score'] > 95]
mapping_nombres = dict(zip(df_correcciones['nombre_salario'], df_correcciones['match_common']))


In [32]:
df_salarios['clean_name'] = df_salarios['clean_name'].replace(mapping_nombres)


In [ ]:
merged_corregido = common_player_info[['person_id', 'display_first_last', 'clean_name']].merge(
    df_salarios[['clean_name', 'temporada', 'salario']],
    on='clean_name',
    how='left'
)

In [34]:
salarios_final = merged_corregido.pivot_table(
    index=['person_id', 'display_first_last'],
    columns='temporada',
    values='salario',
    aggfunc='first'
).reset_index()



In [35]:
print("Jugadores con salario ahora:", salarios_final['display_first_last'].nunique())


Jugadores con salario ahora: 1133


In [36]:
df_correcciones_90 = df_sugerencias[df_sugerencias['score'] > 90]
mapping_nombres_90 = dict(zip(df_correcciones_90['nombre_salario'], df_correcciones_90['match_common']))


In [37]:
df_salarios['clean_name'] = df_salarios['clean_name'].replace(mapping_nombres_90)


In [38]:
merged_corregido_90 = common_player_info[['person_id', 'display_first_last', 'clean_name']].merge(
    df_salarios[['clean_name', 'temporada', 'salario']],
    on='clean_name',
    how='left'
)

salarios_final = merged_corregido_90.pivot_table(
    index=['person_id', 'display_first_last'],
    columns='temporada',
    values='salario',
    aggfunc='first'
).reset_index()


In [39]:
print("Jugadores con salario con score > 90:", salarios_final['display_first_last'].nunique())


Jugadores con salario con score > 90: 1149


In [40]:
df_sugerencias[(df_sugerencias['score'] > 85) & (df_sugerencias['score'] <= 90)].sort_values(by='score', ascending=False).head(20)


,nombre_salario,match_common,score


In [41]:
nombres_salario_actualizados = set(df_salarios['clean_name'].unique())
nombres_final = set(salarios_final['display_first_last'].apply(normalizar_nombre).unique())

nombres_perdidos_nuevos = nombres_salario_actualizados - nombres_final

df_no_unidos = df_salarios[df_salarios['clean_name'].isin(nombres_perdidos_nuevos)][['jugador']].drop_duplicates()
df_no_unidos.to_csv("jugadores_no_unidos.csv", index=False)


In [42]:
player['display_first_last'] = player['first_name'] + ' ' + player['last_name']
player['clean_name'] = player['display_first_last'].apply(normalizar_nombre)


In [43]:
merged_player = player[['id', 'display_first_last', 'clean_name']].merge(
    df_salarios[['clean_name', 'temporada', 'salario']],
    on='clean_name',
    how='left'
)


In [44]:
salarios_final_player = merged_player.pivot_table(
    index=['id', 'display_first_last'],
    columns='temporada',
    values='salario',
    aggfunc='first'
).reset_index()


In [45]:
print("Jugadores con salario usando tabla player:", salarios_final_player['display_first_last'].nunique())


Jugadores con salario usando tabla player: 1313


In [46]:
# Normalizamos nombres para comparar correctamente
nombres_salario = set(df_salarios['clean_name'].unique())
nombres_unidos = set(salarios_final_player['display_first_last'].apply(normalizar_nombre).unique())

# Jugadores con salario que aún no están en la tabla final
nombres_faltantes = nombres_salario - nombres_unidos

print("Jugadores con salario aún no unidos:", len(nombres_faltantes))


Jugadores con salario aún no unidos: 131


In [47]:
faltantes_df = df_salarios[df_salarios['clean_name'].isin(nombres_faltantes)][['jugador', 'clean_name']].drop_duplicates()
faltantes_df.to_csv("jugadores_faltantes_con_salario.csv", index=False)


In [48]:
nombres_player_base = player['clean_name'].unique().tolist()


In [50]:
from rapidfuzz import process, fuzz

sugerencias_nuevas = []

for nombre in nombres_faltantes:
    match, score, _ = process.extractOne(
        nombre,
        nombres_player_base,
        scorer=fuzz.token_sort_ratio
    )
    if score > 85:  # probamos con umbral bajo para ampliar
        sugerencias_nuevas.append((nombre, match, score))

df_sugerencias_nuevas = pd.DataFrame(sugerencias_nuevas, columns=['nombre_salario', 'match_player', 'score'])
df_sugerencias_nuevas = df_sugerencias_nuevas.sort_values(by='score', ascending=False)
df_sugerencias_nuevas.head(30)


,nombre_salario,match_player,score
45,gary trent jr,gary trent jr.,96.296296
35,pj washington,p.j. washington,92.857143
56,cj williams,c.j. williams,91.666667
42,juan hernangomez,juancho hernangomez,91.428571
13,j.d. davison,jd davison,90.909091
3,cj watson,c.j. watson,90.000000
8,dj wilson,d.j. wilson,90.000000
7,tj warren,t.j. warren,90.000000
0,pj tucker,p.j. tucker,90.000000
30,troy brown,tony brown,90.000000


In [51]:
df_correcciones_nuevas = df_sugerencias_nuevas[df_sugerencias_nuevas['score'] > 90]
mapping_nuevos_nombres = dict(zip(df_correcciones_nuevas['nombre_salario'], df_correcciones_nuevas['match_player']))


In [52]:
df_salarios['clean_name'] = df_salarios['clean_name'].replace(mapping_nuevos_nombres)


In [53]:
merged_player_actualizado = player[['id', 'display_first_last', 'clean_name']].merge(
    df_salarios[['clean_name', 'temporada', 'salario']],
    on='clean_name',
    how='left'
)

salarios_final_player = merged_player_actualizado.pivot_table(
    index=['id', 'display_first_last'],
    columns='temporada',
    values='salario',
    aggfunc='first'
).reset_index()


In [54]:
print("Jugadores con salario después de aplicar nuevas correcciones:", salarios_final_player['display_first_last'].nunique())


Jugadores con salario después de aplicar nuevas correcciones: 1318


In [64]:
coincidencias_85 = df_sugerencias_nuevas[df_sugerencias_nuevas['score'] > 85]
print("Coincidencias con score > 85:", coincidencias_85.shape[0])


Coincidencias con score > 85: 57


In [ ]:
# Normalizamos los nombres unidos en la tabla final
nombres_unidos = set(salarios_final_player['display_first_last'].apply(normalizar_nombre))

# Nombres de salario original
nombres_salario = set(df_salarios['clean_name'].unique())

# Jugadores que todavía no se unieron
faltantes = nombres_salario - nombres_unidos

print("Jugadores con salario aún no unidos a player:", len(faltantes))
                                                

Jugadores con salario aún no unidos a player: 126


In [66]:
df_faltantes = df_salarios[df_salarios['clean_name'].isin(faltantes)][['jugador']].drop_duplicates()
df_faltantes.head(40)


,jugador
38,Nenê
84,OJ Mayo
111,Hidayet Turkoglu
151,Enes Kanter
153,Jose Juan Barea
166,Otto Porter
238,Maurice Williams
263,Marcus Morris
266,CJ Watson
276,Moe Harkless


In [68]:
coincidencias_intermedias = df_sugerencias_nuevas[
    (df_sugerencias_nuevas['score'] > 85) & (df_sugerencias_nuevas['score'] <= 90)
]
print("Coincidencias con score entre 85 y 90:", coincidencias_intermedias.shape[0])
coincidencias_intermedias.head(52)


Coincidencias con score entre 85 y 90: 52


,nombre_salario,match_player,score
3,cj watson,c.j. watson,90.000000
8,dj wilson,d.j. wilson,90.000000
7,tj warren,t.j. warren,90.000000
0,pj tucker,p.j. tucker,90.000000
30,troy brown,tony brown,90.000000
39,cj wilcox,c.j. wilcox,90.000000
25,jp macura,j.p. macura,90.000000
48,pj dozier,p.j. dozier,90.000000
18,aj lawson,a.j. lawson,90.000000
21,dj steward,dj stewart,90.000000


In [69]:
salarios_final_player.to_csv("salarios_nba_final_player.csv", index=False)


In [74]:
correcciones_88_90 = df_sugerencias_nuevas[
    (df_sugerencias_nuevas['score'] >= 88) & (df_sugerencias_nuevas['score'] <= 90)
]

print("Coincidencias con score entre 88 y 90:", correcciones_88_90.shape[0])
correcciones_88_90.head(50)


Coincidencias con score entre 88 y 90: 21


,nombre_salario,match_player,score
3,cj watson,c.j. watson,90.000000
8,dj wilson,d.j. wilson,90.000000
7,tj warren,t.j. warren,90.000000
0,pj tucker,p.j. tucker,90.000000
30,troy brown,tony brown,90.000000
39,cj wilcox,c.j. wilcox,90.000000
25,jp macura,j.p. macura,90.000000
48,pj dozier,p.j. dozier,90.000000
18,aj lawson,a.j. lawson,90.000000
21,dj steward,dj stewart,90.000000


In [75]:
mapping_88_90 = dict(zip(correcciones_88_90['nombre_salario'], correcciones_88_90['match_player']))
df_salarios['clean_name'] = df_salarios['clean_name'].replace(mapping_88_90)


In [76]:
merged_player_actualizado = player[['id', 'display_first_last', 'clean_name']].merge(
    df_salarios[['clean_name', 'temporada', 'salario']],
    on='clean_name',
    how='left'
)

salarios_final_player = merged_player_actualizado.pivot_table(
    index=['id', 'display_first_last'],
    columns='temporada',
    values='salario',
    aggfunc='first'
).reset_index()


In [77]:
print("Jugadores con salario tras score > 88 y ≤ 90:", salarios_final_player['display_first_last'].nunique())


Jugadores con salario tras score > 88 y ≤ 90: 1337


In [78]:
salarios_final_player.to_csv("salarios_nba_final_player.csv", index=False)
